Yahooニュースをスクレイピングする関数を定義したので、シェアします。  
データフレームを返してくれるので、データ分析に使えます！！  

requests、BeautifulSoupを利用しています。  

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import unicodedata
import math
import string

In [2]:
# 記号文字は分析をするにあたって邪魔になるため、記号を取り除く関数を定義します。
# 下のYahooNews関数で使用します。
def symbol_removal(soup):
    soup = unicodedata.normalize("NFKC", soup)
    exclusion = "「」『』【】〈〉《》≪≫、。・◇◆■●" + "\n" + "\r" + "\u3000" # 除去する記号文字を指定
    soup = soup.translate(str.maketrans("", "", string.punctuation  + exclusion))
    return soup


# Yahooニュースをスクレイピングする関数です。
# 引数で指定した数の記事をとってきてデータフレームを返します。
def YahooNews(n=30):
    url = "https://news.yahoo.co.jp/topics/top-picks"
    URL = "https://news.yahoo.co.jp/"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    all_page_links = []
    all_page_links.append(url)
    all_links = []
    while True:
        try:
            next = soup.find("li", class_="pagination_item-next").find("a")["href"]
            next_link = URL + next
            all_page_links.append(next_link)
            next_res = requests.get(next_link)
            soup = BeautifulSoup(next_res.text, "html.parser")
        except:
            break
            
    title_list = []
    category_list = []
    text_list = []
    for url in all_page_links: # all_page_links: 全てのニュースのリスト
            res = requests.get(url) # url: 25個分のニュースのリスト
            soup = BeautifulSoup(res.text, "html.parser")
            page_soup = soup.find_all("a", class_="newsFeed_item_link")
            for href in page_soup:
                link = href["href"] # link: 一つのニュースのリンク(本文は一部のみ)
                all_links.append(link)
    
    if len(all_links) <= n:
        n = len(all_links)
    
    i = 0
    for link in all_links:
        link_res = requests.get(link)
        href_soup = BeautifulSoup(link_res.text, "html.parser")
        try:
            title = href_soup.find("h1", class_=re.compile("^sc")).string
        except:
            continue
        title_link = href_soup.find("a", class_="sc-fUKxqW")["href"] # title_link: 本文
        res = requests.get(title_link)
        soup = BeautifulSoup(res.text, "html.parser")

        category = soup.find_all("li", class_="current")
        try:
            category = category[1].string
        except:
            continue
        else:
            for tag in soup.find_all(["a"]):
                tag.decompose()
            try:
                soup = soup.find("div", class_="article_body").get_text()
                soup = symbol_removal(soup)
                
                text_list.append(soup)
                title_list.append(title)
                category_list.append(category)
                i += 1 # 本文が正常に保存できたことをトリガーにしてカウントを一つ増やすことにします。
                pro_bar = ('=' * math.ceil(i / (n / 20))) + (' ' * int((n / (n / 20)) - math.ceil(i / (n / 20))))
                print('\r[{0}] {1}記事'.format(pro_bar, i), end='')
                if i >= n:
                    df = pd.DataFrame({'title': title_list, 'category': category_list, 'text': text_list})
                    return df
            except:
                continue
    df = pd.DataFrame({'title': title_list, 'category': category_list, 'text': text_list})
    return df

では、YahooNews関数を利用して記事をデータフレームに格納しましょう

In [3]:
df = YahooNews(1000)

[====================] 517記事

In [4]:
df

,title,category,text
0,損? あざとかわいい吉岡里帆,エンタメ,女優吉岡里帆27の2年ぶり2冊目の写真集里帆採取 by Asami Kiyokawa集英社...
1,鬼滅の「聖地」潤う観光地,経済,コロナ禍にもかかわらず異例の大ヒットを記録している劇場版鬼滅の刃 無限列車編映画にとどまらず...
2,電通G コロナで営業利益半減,経済,電通グループが10日発表した2020年19月期連結決算国際会計基準は売上高に当たる収益が9...
3,香港民主派議員、全員辞職,国際,12 北京共同新華社電によると中国の全人代常務委員会会議は11日香港立法会定数70の議員資格...
4,専門家組織 全国的に感染増加,国内,新型コロナウイルス対策を助言する厚生労働省の専門家組織アドバイザリーボードの会合が11日開...
...,...,...,...
512,ホワイトハウス周辺で4人刺傷,国際,米NBCテレビなどによると米大統領選でトランプ大統領とバイデン前副大統領の双方の支持者が集...
513,一方的な勝利宣言 米の反応は,国際,FNNプライムオンラインアメリカ大統領選挙の投票日から一夜明けた4日午前10時前のワシントン...
514,NY株続伸、一時600ドル超高,国際,ニューヨーク時事開票作業が進む米大統領選でトランプバイデン両候補の大接戦が続く中4日午前の...
515,セガサミー ゲーセン運営撤退,経済,セガサミーホールディングスは4日娯楽施設を運営する連結子会社セガエンタテインメント東京の株...


In [5]:
df.to_csv('./YahooNews.csv', index=False)

### 工夫した点：  
- 引数を受け取って必要な数だけ記事をとってくることができるようにしました。
- 進捗が確認できるようにしました。(取って来る記事数が多いと、かなり時間がかかるので。)
- 引数に1000などの大きな数字を入れておくことで、Yahooニュースの記事を全て取得することができます。(yahooニュースは約500記事程度。)
- 記号を除去する関数も実装し、YahooNews関数に組み込んでいます。

### 妥協した点:   
- たまーに複数ページに渡って書かれている記事があるが、その場合は最初の1ページだけとってくる仕様にしております。
- コードがすごく長くなってしまいました。。もっと短くしたいです。
- 最初に全てのページの記事のリンクを取得するようにしているので、引数に指定した記事数が少なくても、最初に30秒程時間がかかりますし、その間は進捗も表示されません。